# Notebook 3: Option Trading with ML
## Feature Selection
What could possibly go wrong?
5/10/2020

Feature Selection and Reduction

Feature selection and reduction is a critical step in designing a machine learning project.

Bad feature sets cause many problems.
Overfitting
Slower training and testing

Good feature sets create many benefits.
Fast training and testing.
Better human insights

Several methods of feature selection and reduction.
Univariate Testing
Correlation table with heat map - seaborn and cor
Testing feature combinations with a Learner


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

data = pd.read_csv("D://Blogs//train.csv")

#read X_train
#read y_train

X = data.iloc[:,0:20]  #independent columns
y = data.iloc[:,-1]    #target column i.e price range
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features



In [ ]:
# Seaborn heat map
import seaborn as sns

import pandas as pd
import numpy as np
import seaborn as sns
data = pd.read_csv("D://Blogs//train.csv")
X = data.iloc[:,0:20]  #independent columns
y = data.iloc[:,-1]    #target column i.e price range
#get correlations of each features in dataset
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn")


In [ ]:
# Recursive Feature Elimination
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
estimator = SVR(kernel="linear")
selector = RFE(estimator, 5, step=1)
selector = selector.fit(X, y)
selector.support_   # selected features
array([ True,  True,  True,  True,  True, False, False, False, False,
       False])
selector.ranking_ # feature rank
